In [ ]:
import os
from dotenv import load_dotenv
from pprint import pprint

from stravalib import Client

load_dotenv(override=True)

In [ ]:
client = Client(
    access_token=os.getenv("STRAVA_ACCESS_TOKEN"),
    token_expires=int(os.getenv("STRAVA_TOKEN_EXPIRES")),
    refresh_token=os.getenv("STRAVA_REFRESH_TOKEN"),
)
athlete = client.get_athlete()

In [ ]:
# https://stravalib.readthedocs.io/en/latest/reference/api/stravalib.model.DetailedAthlete.html#stravalib.model.DetailedAthlete
print(athlete.id)
print(athlete.firstname)
print(athlete.lastname)
print(athlete.sex)
print(athlete.athlete_type)
print(athlete.follower_count)
print(athlete.bikes)
print(athlete.shoes)

In [ ]:
# athlete.model_dump()

In [ ]:
# https://stravalib.readthedocs.io/en/latest/reference/api/stravalib.model.AthleteStats.html#stravalib.model.AthleteStats
print(athlete.stats.all_run_totals)
print(athlete.stats.ytd_run_totals)
print(athlete.stats.recent_run_totals)
print(athlete.stats.biggest_ride_distance)

In [ ]:
athlete.stats.model_dump()

In [ ]:
activities = client.get_activities(limit=15)

In [ ]:
for activity in activities:
    print(f"{activity.name} | {activity.kudos_count} | {activity.distance} | {activity.average_heartrate}")

In [ ]:
for activity in activities:
    print(activity.model_dump())
    break

In [ ]:
import polars as pl

data = []
for activity in activities:
    data.append(activity.model_dump())

df = pl.DataFrame(data)
df

In [ ]:
df.columns

In [ ]:
df.get_column("sport_type").unique()

In [ ]:
for row in df.iter_rows():
    print(row[22])
    # print(row[26])
    # test = row
    break

In [ ]:
df.get_column("map").first()

In [ ]:
import polyline

decoded_polyline = polyline.decode(df.get_column("map").first()["summary_polyline"])
decoded_polyline

In [ ]:
import plotly.graph_objects as go

lat = [point[0] for point in decoded_polyline]
lon = [point[1] for point in decoded_polyline]

min_lat, min_lon, max_lat, max_lon = min(lat), min(lon), max(lat), max(lon)
center_lat = min_lat + (max_lat - min_lat) / 2
center_lon = min_lon + (max_lon - min_lon) / 2

fig = go.Figure()
fig.add_trace(
    go.Scattermap(
        lat=lat,
        lon=lon,
        mode="lines",
        marker={
            "size": 5,
            "color": "#FFA800",
        },
    )
)
fig.update_layout(
    margin={"l": 0, "t": 0, "b": 0, "r": 0},
    map={
        "center": {"lon": center_lon, "lat": center_lat},
        "style": "open-street-map",
        "zoom": 10,
    },
    title={"text": "test"},
)
fig

In [ ]:
# client.get_athlete_stats()
# client.get_athlete_zones()

In [ ]:
for kom in client.get_athlete_koms(athlete.id, limit=3):
    pprint(kom.model_dump())

In [ ]:
streams = client.get_activity_streams(16992333182, ["time", "distance", "latlng", "velocity_smooth", "altitude", "heartrate", "grade_smooth", "cadence"])

In [ ]:
streams

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=streams["distance"].data, y=streams["altitude"].data))
# fig.add_trace(go.Scatter(x=streams["distance"].data, y=streams["cadence"].data))
# fig.add_trace(go.Scatter(x=streams["distance"].data, y=streams["grade_smooth"].data))
fig.add_trace(go.Scatter(x=streams["distance"].data, y=streams["velocity_smooth"].data))
fig

In [ ]:
import numpy as np
import plotly.graph_objects as go

lat = [point[0] for point in streams["latlng"].data]
lon = [point[1] for point in streams["latlng"].data]

fig = go.Figure()
fig.add_trace(
    go.Scattermap(
        lat=lat,
        lon=lon,
        mode="markers+lines",
        # marker={
        #     "size": 5,
        #     "color": "#FF0000",
        # },
        marker_color=streams["velocity_smooth"].data,
        marker_size=5,
        line_color="#FFA800",
    )
)
fig.update_layout(
    margin={"l": 0, "t": 0, "b": 0, "r": 0},
    map={
        "center": {"lon": np.mean(lon), "lat": np.mean(lat)},
        "style": "open-street-map",
        "zoom": 10,
    },
    title={"text": "test"},
)
fig

In [ ]:
import numpy as np
import folium

lats = [point[0] for point in streams["latlng"].data]
lons = [point[1] for point in streams["latlng"].data]
speeds = streams["velocity_smooth"].data
eles = streams["altitude"].data
center_lat = np.mean(lats)
center_lon = np.mean(lons)


m = folium.Map([center_lat, center_lon], zoom_start=15)

color_line = folium.ColorLine(
    positions=list(zip(lats, lons)),
    colors=eles,
    colormap=["y", "orange", "r"],
    weight=5,
).add_to(m)
m

In [ ]:
m.get_root().render()